# Work Plan

* Create LRG catalogue from the halo catalogue using HAM with parameter $\sigma_{HAM}$
    * 2PCF and RSD tests
* Determine $\sigma_{HAM}$ by fitting the HAM model with observation
    * 2PCF, $\chi^2$ minimise
        * the code test for $\chi^2$ using primary simulation and observations (just for tests)
        * 2PCF fibre collision effects determination using EZmock(maybe directly from others)  
        * (discarded)Jackknife method to produce 2pcf observation errors in small scale
        * get the LRG distribution function in haloes
* Do the same for ELG but with more complicated HAM model
    * 3 free parameters in HAM models
    * get the ELG distribution function in haloes 
    

---------------------------------------------------------------------------------------------------------------------------------------------
* Cross-correlate the LRG and ELG
    * Produce the ELG and LRG catalogue simultaneously according to the galaxy distribution function
    * count the galaxy pairs and calculate the 2pcf
    * Fit the LRG 2PCF, ELG 2PCF and LRG-ELG CCF with observation
        * parameters: 1(ELG-LRG overlap)

## Process
### <font color="#dd0000">To do</font><br /> 
* solve the problem: 
    * interactive nodes out of memory
    * reproduce the LRG and ELG results
* determine the outline
* write the cosmological background (introduction)
* write the technical details
* the acknowledge and abstract

### Done
* read test halo catalogue and created LRG catalogues from it 
    * scattering parameter $\sigma_{HAM}$
* Calculated the mono-, quadru- and hexadeca-poles of the 2-point correlation function
    * used 2PCF code from Cheng
* converted the halo/galaxy catalogue from the real space to the redshift space
    * RSD effect
* determined the starting point of 2PCF
    * start from 0 is acceptable
    * fibre collision effect are so small (<1Mpc/h) that our bin size cannot resolute it
* used the Corrfunc to count the galaxy pairs for $\chi^2$ tests
    * ensured Corrfunc could produce the same result as 2pcf
    * Corrfunc.theory.DDsmu() and np.trapz() were employed to calculate multipoles
    * RR analytical form should be $\frac{4\pi}{3}\frac{s_{max}^3-s_{min}^3}{nmu}$
* observation and covariance matric for $\chi^2$ tests
    * observation use 2pcf, bin width = 1Mpc/h, multipoles:[mono,quad]
    * covariance matrix from the existing DD,DR,RR files
        * pair counts results match the 2pcf results of mocks
        * np.trapz() used
        * it is a covariance matrix of multipoles' combination, i.e., cov = np.cov([mono,quad]) 
        * unbiased $C^{-1}=np.linalg.pinv(cov)\times \frac{N_{bins}-N_{mocks}-2}{N_{bins}-1}$
* completing the $\chi^2$ calculation code
    * observation 2pcf multipoles
    * mock 2pcf multipoles and covariance matrix
    * Normalised RR analytical calculation
    * <font color="#dd0000">(fixing)creating random numbers with multiple seeds and construct galaxy catalogues and produce the correlation functions</font><br /> 
    * model = average over the correlation functions(<font color="#dd0000">only the monopoles</font><br/>) for a given sigma with different seeds
    * $\chi^2$ minimising
    * $\xi_{l}(s)$ of the best model comparing with the observation
    * optimising it to make it faster and more robust.
        * speed up the code by generating uniform random numbers and inverse transform it into normal random numbers
* test the LRG and ELG versions:]
    * $\chi^2-parameters$ distribution to select the suitable parameter range and the step size
* Wait for the real data for the galaxy distribution determination
* Do the same for ELGs


### **solved problems**:
* LRG sorting process takes lots of time
    * use np.argpartition instead of np.sort (no need to sort, we just want Nth maximum)
* the same seed and sigma provide different 2pcf results(inconsistency)
    * specify the random seed in the $\chi^2$ function
* how to plot error bars
    * yerr = np.std(mockarray,axis=1)
* **removed the s=0 bin from the fitting or smin>1Mpc/h** 
    * s=0 heavily contributed to $\chi^2$ thus resulted in rediculous conclusion drawn from $\chi^2$
* find an appropriate $\chi2$ minimiser:
    * Minute.iminuit: sigma.migrad(precision=0.001) helps converge in <300s and produce sigma with smaller $\chi^2$ than scipy.optimise.minimize(takes ~150s)
* parallel calculation
    * multiprocessing.pool.map for single-variable functions and multiprocessing.pool.starmap for nultiple-variable functions
* time-consuming np.random.normal()
    * replaced by np.random.uniform with Box-Muller Transformation.
* len(catalogue)\*2 exceeds the maximum capability of the function
    * only produce uniform random numbers with length = len(catalogue)
* incorrect ELG prior: 
    * the vmax cut in the high end should be as low as ~100km/s instead of 1000km/s.
        * because LRG cut ~400km/s,$M_{LRG}\approx 100M_{ELG}$ , then $v \propto \sqrt{M}$, so the prior should be (0,200)